In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins

In [2]:
df = pd.read_excel("../original_data/private_institute.xlsx")

In [3]:
df_code = pd.read_excel("../original_data/district_code_seoul.xlsx")
df_land = pd.read_csv("../original_data/land_price.csv", engine = 'python')

In [4]:
df_code = df_code.rename(columns = {'통계청행정동코드' : 'code', '행자부행정동코드':'code_mois', '시도명':'city', '시군구명':'district', '행정동명': 'village'})
df_code = df_code.drop([0])
df_code["code"] = df_code["code"].astype('str')
df_code["code_district"] = df_code["code"].str[0:5]
df_code = df_code.drop_duplicates(['district'])
df_code = df_code.drop(['city','village', 'code', 'code_mois'], axis = 1)

In [5]:
df =df[['district', 'university related institute']]
df = df.drop([0])

df = pd.merge(df, df_code, how = "left")

In [6]:
state_geo = '../original_data/skorea_municipalities_geo_simple.json'

with open(state_geo, encoding='utf-8') as f:
    json_data = json.load(f)

#Error solution: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [7]:
df_land = df_land.rename(columns = {'시군구명': 'district', '필지구분명': 'land_type',  '공시지가(원/㎡)':'land_price'})
df_land = df_land[df_land["land_type"]=="토지"]
df_price = pd.DataFrame(df_land.groupby("district")["land_price"].mean()).reset_index()
df_price = pd.merge(df, df_land, how = "left")

In [8]:
m = folium.Map(location=[37.564214, 127.001699], tiles="OpenStreetMap", zoom_start=11)
#stamentoner

m.choropleth(
    geo_data=json_data,
    data=df,
    columns=['district', 'university related institute'],
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.3,
    color = 'gray',
    key_on = 'feature.id'
)
m

C:\Users\thxkn\anaconda3\lib\site-packages\folium\folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [9]:
m = folium.Map(location=[37.564214, 127.001699], tiles="OpenStreetMap", zoom_start=11)
#stamentoner

m.choropleth(
    geo_data=json_data,
    data=df_price,
    columns=["district", "land_price"],
    fill_color='PuRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    color = 'gray',
    key_on = 'feature.id'
)
m

In [18]:
# 땅값(주택용 토지 값) = 부모님 income proxy, distric별 순위 매기기
# 학원 수와 땅 값의 상관관계
#학원 수 = 부모님 소득 * beta + control (유동인구, )
import io
from PIL import Image
import selenium

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


In [86]:
def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

In [87]:
center_locations = pd.DataFrame()
codes = []
names = []
x_list = []
y_list = []

for x in range(0, len(json_data['features'])):
    code = json_data['features'][x]['properties']['EMD_CD']
    name = json_data['features'][x]['properties']['EMD_KOR_NM']
    # 중앙값 생성
    points = json_data['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    # 결과
    codes.append(code)
    names.append(name)
    x_list.append(X)
    y_list.append(Y)

# 데이터 프레임 생성
center_locations['CODE'] = codes
center_locations['NAME'] = names
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [91]:
center_locations2 = center_locations[center_locations['Y'] >= 37.426026]
target_df = pd.merge(df,center_locations2, how = 'left')
target_df = target_df.dropna(axis=0, subset=['X','Y']) # 위치 정보 없는 값 제외

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [ ]:
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)

    if target_df.iloc[i]['NAME'] in ['서초구','강남구'] :
        color = 'white'
    else:
        color = '#3186cc'

    folium.CircleMarker(location, radius=10,color=color,fill_color=color,fill_opacity = 0.1, opacity=0.0,  popup=target_df.iloc[i]['NAME'] + "\n" + str(int(round(target_df.iloc[i]['VALUE']/10000,0))) + "만원").add_to(m)


folium.LayerControl(collapsed=False).add_to(m)